# Week 04a Assignment weather data

Welcome to week four of this course programming 1. Analyzing time related data such as estimating seasonal effect, or year effect might be a challenge. How to filter the essential information from the noise? How to apply signal analysis with noisy data. How to make compact useful visualizations? Python has several constructs to handle date time related data. The relevant classes for making plots are Locators and Formatters. Locators determine where the ticks are, and formatters control the formatting of tick labels. The relevant class for date time data is the pandas datetime data type, which has methods like resample and several possibilities to display data (frequencies). As a study case we will work with weather data. If you have data that fits the learning goals, you can bring your own data.

Keywords: signal processing, smoothing, resample, formatters and locators, datetime object

More to read: 

- https://fennaf.gitbook.io/bfvm22prog1/
- https://machinelearningmastery.com/time-series-data-visualization-with-python/
- https://towardsdatascience.com/how-to-plot-time-series-86b5358197d6
- In the https://pandas.pydata.org/docs/reference/offset_frequency.html you can find more about frequencies and in the documentation
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html you can read all the methods of this datetime object.
- https://en.wikipedia.org/wiki/Smoothing


Learning objectives

- load, inspect and clean a dataset
- reshape dataframes to group data in a certain frequency
- apply smoothing technologies
- Create useful visualisation with timeseries data
- Maintain development environment 
- Apply coding standards and FAIR principles

Please add topics you want to learn here: https://padlet.com/ffeenstra1/z9duo25d39dcgezz


## Assignment

You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below. 

<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. widgets selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
Mind you that you cannot copy code without referencing the code. If you copy code you need to be able to explain your code verbally and you will not get the full score. 


NB if you want to make a plot with more actual data you can download data from https://openweathermap.org/api 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. 
Preferably we read the data not with a hard coded data path but using a config file. See https://fennaf.gitbook.io/bfvm22prog1/data-processing/configuration-files/yaml. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from a station. Station 270 is in the neighborhood of Groningen. For our plot we only need the the mean, minimum and maximum temperature. Of course you are welcome to select other data if you think it might be useful for your visualization. The data should look something like this:


In [1]:
import pandas as pd
import numpy as np
import yaml

#make a config file
with open('config.yaml','r') as stream:
    config = yaml.safe_load(stream)
weather = config['weather']
read_weather = pd.read_csv(weather, skiprows=65)
df_weather = pd.DataFrame(read_weather)

df_weather.drop(df_weather.columns[[5,6,7]], axis=1, inplace=True) #droping unneccessary columns
df_weather.columns = ['station','date', 'Tmean','Tmin','Tmax'] #creating head to each column
df_270 = df_weather[df_weather['station']==270]


C:\Users\Mahdiye\AppData\Local\Temp\ipykernel_9548\2581276118.py:9: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  read_weather = pd.read_csv(weather, skiprows=65)


---

<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [2]:
#replace cells with spaces to NaN
df_270 = df_270.replace(r'^\s*$', np.nan, regex=True)

#change date formats
df_270['date'] = pd.to_datetime(df_270['date'], format='%Y%m%d')

#change temperatures to celcius degrees
df_270['Tmean'] = (df_270['Tmean'].astype('int')/10)
df_270['Tmin'] = (df_270['Tmin'].astype('int')/10)
df_270['Tmax'] = (df_270['Tmax'].astype('int')/10)

#remove leap year
df_270['LeapYear'] = df_270['date'].dt.is_leap_year
df_270 = df_270[df_270['LeapYear']==False]

df_270.index = df_270['date']
df_270.head()


,station,date,Tmean,Tmin,Tmax,LeapYear
date,,,,,,
2001-01-01,270,2001-01-01,2.1,0.4,3.8,False
2001-01-02,270,2001-01-02,6.3,3.7,8.5,False
2001-01-03,270,2001-01-03,5.3,2.6,7.9,False
2001-01-04,270,2001-01-04,6.5,4.6,7.9,False
2001-01-05,270,2001-01-05,6.6,5.6,8.3,False


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [3]:
#Test your outcome
#write code to check if you have done the above

### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [4]:
#split to 2 dataframes, before and after 2018
split_date ='2018-01-01'
df_before_2018 = df_270.loc[df_270['date'] < split_date]
df_after_2018 = df_270.loc[df_270['date'] >= split_date]

In [5]:
df_multipleyears = df_before_2018
df_multipleyears.index = df_multipleyears['date']
df_groupedbymonthday = df_multipleyears.groupby(by=[df_multipleyears.index.month, df_multipleyears.index.day])['Tmin'].min()
df_groupedbymonthday.index.names = ['month', 'day']

In [6]:
def month_day(df_multipleyears = df_before_2018, min_max = 'Tmin'):
    if min_max=='Tmin':
            df_groupedbymonthday = df_multipleyears.groupby(by=[df_multipleyears.index.month, df_multipleyears.index.day])[min_max].min()
    elif min_max=='Tmax':
            df_groupedbymonthday = df_multipleyears.groupby(by=[df_multipleyears.index.month, df_multipleyears.index.day])[min_max].max()

    df_groupedbymonthday.index.names = ['month', 'day']
    return df_groupedbymonthday

In [7]:
print(month_day(df_multipleyears = df_before_2018, min_max = 'Tmin'))

month  day
1      1      -5.8
       2      -7.5
       3     -12.6
       4      -6.7
       5      -6.2
              ... 
12     27     -6.0
       28     -7.4
       29     -7.3
       30     -6.7
       31     -5.1
Name: Tmin, Length: 365, dtype: float64


In [8]:
#Test your code
def test_reformed(df = df_270, min_max='Tmin'):
    #
    df = df[(df.index.year > 2007) & (df.index.year < 2018)]
    df_groupedbymonthday = month_day(df,min_max)

    return df_groupedbymonthday

test_reformed(df= df_270, min_max='Tmin')

month  day
1      1      -5.8
       2      -7.5
       3     -12.6
       4      -4.1
       5      -6.0
              ... 
12     27     -2.0
       28     -4.2
       29     -5.0
       30     -6.7
       31      0.2
Name: Tmin, Length: 365, dtype: float64

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm. Ask the signal analysis teacher if you want some advice.


In [9]:
#different types of interpolations
from scipy import interpolate
from scipy.interpolate import UnivariateSpline

def interpolated_method(x,y):
    smoothed1 = interpolate.interp1d(x, y)
    y_new1 = smoothed1(x)
    
    smoothed2 = interpolate.splrep(x, y, s=0)
    y_new2 = interpolate.splev(x, smoothed2, der=0)
    
    smoothed3 = UnivariateSpline(x, y)
    y_new3 = smoothed3(x)
    
    
    return y_new1, y_new2, y_new3
    

---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

In [10]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.plotting import ColumnDataSource
from bokeh.io import output_notebook
from bokeh.models import Band
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter

output_notebook()

Loading BokehJS ...

In [11]:
#convert Tmin and Tmax to an array
min_multiyears = month_day(df_multipleyears = df_before_2018, min_max = 'Tmin')
min_multiyears_array = np.array(min_multiyears)

max_multiyears = month_day(df_multipleyears = df_before_2018, min_max = 'Tmax')
max_multiyears_array = np.array(max_multiyears)

In [12]:
#create a DataFrame containing Tmean of the year 2018, and Tmin and Tmax of the years before 2018
df = df_after_2018[['Tmean']]
df['Tmin_multi'] = min_multiyears_array
df['Tmax_multi'] = max_multiyears_array
df = df.reset_index()
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df

C:\Users\Mahdiye\AppData\Local\Temp\ipykernel_9548\413687505.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tmin_multi'] = min_multiyears_array
C:\Users\Mahdiye\AppData\Local\Temp\ipykernel_9548\413687505.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tmax_multi'] = max_multiyears_array


,date,Tmean,Tmin_multi,Tmax_multi
0,2018-01-01,6.0,-5.8,10.1
1,2018-01-02,5.6,-7.5,10.2
2,2018-01-03,7.5,-12.6,10.7
3,2018-01-04,7.3,-6.7,9.8
4,2018-01-05,6.0,-6.2,9.4
...,...,...,...,...
360,2018-12-27,5.7,-6.0,11.7
361,2018-12-28,7.1,-7.4,10.9
362,2018-12-29,8.5,-7.3,8.6
363,2018-12-30,8.0,-6.7,11.1


In [13]:
#plot a line graph by the information of the year 2018, which shaded to min and max of the years before 2018
def render_plot(x = df['date'],int_type='real',y = df['Tmean']):
    p = figure(title='comparing the'+int_type+ ' average of temperature in 2018 with max and min of other years', background_fill_color="white")
    p.line(x, y, color="blue",legend_label = 'temperature mean,2018')
    p.xaxis[0].formatter=DatetimeTickFormatter()
    p.xaxis.axis_label = 'date'
    p.yaxis.axis_label = int_type+' mean'

    source = ColumnDataSource(df)
    band = Band(base = 'date', lower='Tmin_multi', upper = 'Tmax_multi', source=source,
                fill_alpha=0.3, fill_color="gray", line_color="gray")
    p.add_layout(band)

    return(p)
show(render_plot(x = df['date'],int_type='real',y=df['Tmean']))

In [14]:
y_interp1d, y_splrep , y_UnivariateSpline = interpolated_method(x= df.index.values ,y= df['Tmean'] )
df['y_interp1d'] = y_interp1d
df['y_splrep'] = y_splrep
df['y_UnivariateSpline'] = y_UnivariateSpline

---

In [15]:
# plot Tmean of year 2018 and compare with smoothed T_mean
p1 = render_plot(x = df['date'],int_type='real',y=df['Tmean'])
p2 = render_plot(x = df['date'],int_type='interp1d',y=y_interp1d)
p3 = render_plot(x = df['date'],int_type='splrep',y=y_splrep)
p4 = render_plot(x = df['date'],int_type='y_UnivariateSpline',y=y_UnivariateSpline)
p = gridplot([[p1,p2,p3,p4]], width=500, height=300)
show(p)


NOTE: I plotted real Tmean, and then decided to smooth Tmean by different ways. As the figures shown, smoothing Tmean by UnivariateSpline method, is more smoothed than other methods.So, I think this way is better than others to smooth this data.

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Or maybe a widget were you choose a different station. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather

In [16]:
import panel as pn
pn.extension()

In [17]:
columns=df.columns[:]
smooth_types = ['real','interp1d','splrep','y_UnivariateSpline']
inter_plot = pn.interact(render_plot, x = columns, int_type = smooth_types, y = columns)
inter_plot

Column
    [0] Column
        [0] Select(name='x', options=['date', 'Tmean', ...], value='date')
        [1] Select(name='int_type', options=['real', 'interp1d', ...], value='real')
        [2] Select(name='y', options=['date', 'Tmean', ...], value='date')
    [1] Row
        [0] Bokeh(Figure, name='interactive00101')